# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''

import numpy as np
import pandas as pd
reviews = pd.read_csv("IMDB Dataset.csv")

reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
#Split the data into train and split

from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_sentiment, test_sentiment = train_test_split(reviews["review"],reviews["sentiment"],test_size = 0.2)

In [3]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0

train_reviews = pd.DataFrame(train_reviews)
test_reviews = pd.DataFrame(test_reviews)
train_sentiment = pd.DataFrame(train_sentiment)
test_sentiment = pd.DataFrame(test_sentiment)

train_reviews = train_reviews.head(100)
test_reviews = test_reviews.head(100)
train_sentiment = train_sentiment.head(100)
test_sentiment = test_sentiment.head(100)

def change(text):
    if text == "positive":
        return 1
    else:
        return 0

train_sentiment["sentiment"] = train_sentiment["sentiment"].apply(lambda x: change(x))
train_sentiment
test_sentiment["sentiment"] = test_sentiment["sentiment"].apply(lambda x: change(x))

test_sentiment

,sentiment
6596,0
46322,1
8238,1
33572,1
19731,0
...,...
13501,0
32531,1
32214,1
25171,1


In [4]:
train_reviews.head()

,review
28964,"Seriously, Sci-Fi needs to stop making movies...."
28393,wow...I just watched this movie...American peo...
34058,"""Go Fish"" garnered Rose Troche rightly or wron..."
31485,What a waste! This movie could have really bee...
23325,"This movie is really bad, trying to create sci..."


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [5]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
import re
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    text_seqs = re.sub(r'[^\w\s]+',"",text_seqs)
    doc = encoder(text_seqs)
    l1 = []
    for word in doc: 
        l1.append(word.text.lower())
    return l1

#train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews['Tokenized_Review'] = train_reviews['review'].apply(lambda x:text_to_tokens(x))
test_reviews['Tokenized_Review'] = test_reviews['review'].apply(lambda x:text_to_tokens(x))

train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
28964,"Seriously, Sci-Fi needs to stop making movies....","[seriously, scifi, needs, to, stop, making, mo..."
28393,wow...I just watched this movie...American peo...,"[wowi, just, watched, this, movieamerican, peo..."
34058,"""Go Fish"" garnered Rose Troche rightly or wron...","[go, fish, garnered, rose, troche, rightly, or..."
31485,What a waste! This movie could have really bee...,"[what, a, waste, this, movie, could, have, rea..."
23325,"This movie is really bad, trying to create sci...","[this, movie, is, really, bad, trying, to, cre..."
26188,"My 7-year-old daughter loved it, as Disney exe...","[my, 7yearold, daughter, loved, it, as, disney..."
14387,"I am Curious (Yellow) (a film, in near Seussic...","[i, am, curious, yellow, a, film, in, near, se..."
46671,This meandering tale of mob revenge is simply ...,"[this, meandering, tale, of, mob, revenge, is,..."
11180,"This was my very first ""Bollywood"" movie and I...","[this, was, my, very, first, bollywood, movie,..."
17570,After watching some of HBO's great stuff - Ban...,"[after, watching, some, of, hbos, great, stuff..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [6]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle
import gensim
from gensim.utils import simple_preprocess

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    dictionary = gensim.corpora.Dictionary(train_reviews["Tokenized_Review"])
    
    
    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    dictionary.filter_extremes(no_below = min_freq+1)
    token_counts = dictionary.cfs
    lexicon = dictionary

    # lexicon = 
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    # lexicon = 

    list1 = []
    for k,v in lexicon.iteritems():
        list1.append((v,k+2))

    dictionary = dict(list1)
    lexicon = dictionary
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'5': 2, 'acting': 3, 'after': 4, 'along': 5, 'at': 6, 'away': 7, 'brings': 8, 'by': 9, 'ca': 10, 'call': 11, 'cheesy': 12, 'could': 13, 'creature': 14, 'crew': 15, 'daughter': 16, 'dr': 17, 'either': 18, 'escapes': 19, 'figure': 20, 'from': 21, 'go': 22, 'guy': 23, 'had': 24, 'he': 25, 'help': 26, 'his': 27, 'horrible': 28, 'how': 29, 'however': 30, 'incredibly': 31, 'john': 32, 'kill': 33, 'killing': 34, 'looks': 35, 'making': 36, 'many': 37, 'men': 38, 'movies': 39, 'needs': 40, 'or': 41, 'other': 42, 'out': 43, 're': 44, 'scifi': 45, 'seriously': 46, 'ship': 47, 'shoot': 48, 'so': 49, 'some': 50, 'starts': 51, 'stay': 52, 'stop': 53, 'them': 54, 'then': 55, 'there': 56, 'they': 57, 'thing': 58, 'about': 59, 'also': 60, 'another': 61, 'black': 62, 'cinema': 63, 'cinematography': 64, 'content': 65, 'country': 66, 'cover': 67, 'crap': 68, 'dances': 69, 'days': 70, 'different': 71, 'direction': 72, 'director': 73, 'far': 74, 'films': 75, 'friday': 76, 'gangster': 77, 'good': 78, 'highl

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [7]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    id = []
    for x in token_seqs:
        for key,value in lexicon.items():
            if x == key:
                id.append(value)
    idx_seqs = id
    return idx_seqs

#train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
train_reviews['Review_Idxs'] = train_reviews['Tokenized_Review'].apply(lambda y:tokens_to_idxs(y,lexicon))
test_reviews['Review_Idxs'] = test_reviews['Tokenized_Review'].apply(lambda y:tokens_to_idxs(y,lexicon))
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
28964,"[seriously, scifi, needs, to, stop, making, mo...","[46, 45, 40, 53, 36, 39, 57, 44, 28, 24, 32, 2..."
28393,"[wowi, just, watched, this, movieamerican, peo...","[85, 113, 93, 111, 107, 80, 75, 102, 83, 75, 6..."
34058,"[go, fish, garnered, rose, troche, rightly, or...","[22, 125, 126, 141, 41, 155, 91, 55, 127, 147,..."
31485,"[what, a, waste, this, movie, could, have, rea...","[115, 179, 13, 171, 120, 175, 162, 180, 169, 6..."
23325,"[this, movie, is, really, bad, trying, to, cre...","[171, 190, 233, 198, 186, 200, 148, 230, 7, 21..."
26188,"[my, 7yearold, daughter, loved, it, as, disney...","[257, 16, 253, 144, 275, 265, 263, 184, 269, 2..."
14387,"[i, am, curious, yellow, a, film, in, near, se...","[282, 238, 373, 394, 392, 6, 409, 290, 424, 23..."
46671,"[this, meandering, tale, of, mob, revenge, is,...","[455, 99, 110, 348, 124, 456, 449, 451, 444, 4..."
11180,"[this, was, my, very, first, bollywood, movie,...","[257, 110, 332, 464, 477, 172, 150, 37, 42, 50..."
17570,"[after, watching, some, of, hbos, great, stuff...","[4, 235, 50, 555, 163, 604, 460, 523, 528, 540..."


In [8]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = np.bincount(idx_seqs)
    return bow_seqs
    

#bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'],matrix_length=len(lexicon) + 1) #add one to length for padding)
bow_train_reviews = train_reviews['Review_Idxs'].apply(lambda x:idx_seqs_to_bows(x,matrix_length=len(lexicon)+1))

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews)], columns=['Word', 'Count'])

TRAIN INPUT:
 28964    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, ...
28393    [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...
34058    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ...
31485    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
23325    [0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, ...
                               ...                        
19051    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
17731    [0, 0, 0, 1, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0, 0, ...
48755    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8748     [0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, ...
30569    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Review_Idxs, Length: 100, dtype: object
SHAPE: (100,) 



,Word,Count
0,,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, ..."
1,<UNK>,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
3,acting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,after,"[0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, ..."
...,...,...
95,real,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,realistic,"[0, 0, 0, 1, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0, 0, ..."
97,recommended,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,shows,"[0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, ..."


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = train_reviews['Review_Idxs'].str.len().max()
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    return pad_sequences(idx_seqs,maxlen = max_seq_len)


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[   0    0    0 ...   47    3   28]
 [   0    0    0 ...   83   63   59]
 [   0    0    0 ...   49  134   92]
 ...
 [   0    0    0 ...  129  804 1382]
 [   0    0    0 ...   54  705   41]
 [   0    0    0 ...  342  284 1288]]
SHAPE: (100, 509) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [10]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    model = tf.keras.Sequential()
    input_layer = Input(shape=(None,))
    model.add(input_layer)
    # Layer 2
    model.add(Embedding(input_dim=n_input_nodes,output_dim = n_embedding_nodes,mask_zero = True))

    # Layer 3
    model.add(GRU(units = n_hidden_nodes))
    
    # Layer 4
    model.add(Dense(units = 1))
    
    #Specify which layers are input and output, compile model with loss and optimization functions
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model


In [11]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [12]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
train_input = train_padded_idxs
train_output = tf.convert_to_tensor(train_sentiment[['sentiment']].values,dtype = tf.int64)
test_input  = test_padded_idxs
test_output = tf.convert_to_tensor(test_sentiment[['sentiment']].values,dtype = tf.int64)

rnn_model.fit(x =train_input,y=train_output,epochs = 5,batch_size = 20)

Epoch 1/5
5/5 [==============================] - 10s 2s/step - loss: 0.3346 - accuracy: 0.5900
Epoch 2/5
5/5 [==============================] - 8s 2s/step - loss: 0.2844 - accuracy: 0.7200
Epoch 3/5
5/5 [==============================] - 8s 2s/step - loss: 0.1373 - accuracy: 0.6800
Epoch 4/5
5/5 [==============================] - 8s 2s/step - loss: 0.0985 - accuracy: 0.8400
Epoch 5/5
5/5 [==============================] - 8s 2s/step - loss: 0.0323 - accuracy: 0.9700


##  <font color='#6629b2'>Prediction</font>

In [13]:
'''Put test reviews in padded matrix just how we did for train_reviews'''


print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[   0    0    0 ... 1256  460  639]
 [   0    0    0 ...  266  208   49]
 [   0    0    0 ...  398    6   63]
 ...
 [   0    0    0 ... 1351 1542   21]
 [   0    0    0 ...   78  443  189]
 [   0    0    0 ...   88  473   60]]
SHAPE: (100, 509) 



In [14]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = np.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [15]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
score = r2_score(test_reviews['RNN_Pred_Rating'],test_output)
print(score)

-1.7960526315789465


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [16]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (100, 500)
[-7.65265897e-03  2.05201246e-02  3.19134295e-02  2.55749598e-02
  2.64417864e-02  3.35756615e-02  9.38560162e-03  3.65877561e-02
 -2.04797462e-02  1.71994939e-02  2.92575974e-02  2.35721692e-02
  7.47844297e-03  2.36763638e-02  1.45026566e-02  4.01279237e-03
  2.36452334e-02 -4.30870727e-02 -1.62908621e-02 -7.24585131e-02
  1.68756172e-02 -1.01558603e-02 -2.35274881e-02  1.67772342e-02
  2.29001231e-02 -1.46655496e-02 -2.94298194e-02 -3.14796120e-02
 -5.44016156e-03 -1.17722312e-02  1.44683234e-02  3.08129862e-02
  2.80372854e-02 -6.40035979e-03  5.68462769e-03  2.57684384e-02
  2.00733989e-02  1.83834247e-02  8.38497840e-03  4.80674533e-03
  1.95823051e-02 -2.14669341e-03  2.61424519e-02  2.48013567e-02
  1.54345930e-02 -1.99551210e-02  1.07974280e-02  4.41645924e-03
 -2.51103193e-02  1.69071183e-02 -2.21852902e-02  4.72250432e-02
 -2.63532549e-02  1.53796393e-02  2.00189054e-02  7.15101184e-03
 -1.78908743e-02  2.01202556e-02 -2.34364867e-02 

## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.